# BPIC 2017 dataset preparation


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from CI_Experiments.config import PROJECT_DIR

In [ ]:
UNPRAPARED_PATH = PROJECT_DIR / 'data/unprepared_process_logs'
BPIC_2017_DATA = UNPRAPARED_PATH / 'BPIC2017.csv'
DESTINATION_DATA = PROJECT_DIR / 'data/prepared_process_logs/BPIC2017/activity/activity.csv'
NUMERIC_TRAIN = PROJECT_DIR / 'data/prepared_process_logs/BPIC2017/numeric/train.csv'
NUMERIC_TEST = PROJECT_DIR / 'data/prepared_process_logs/BPIC2017/numeric/train.csv'


In [ ]:
unprepared_data = pd.read_csv(BPIC_2017_DATA, sep=';')
unprepared_data.head()

In [ ]:
unprepared_data.columns

In [ ]:
unprepared_data['LoanGoal'].unique()

In [ ]:
unprepared_data['ApplicationType'].unique()

In [ ]:
numeric_columns = [
    'CreditScore',
    'OfferedAmount',
    'NumberOfTerms',
    'MonthlyCost',
    'FirstWithdrawalAmount'
]

In [ ]:
other_columns = ['Case ID', 'label', 'LoanGoal', 'ApplicationType']

In [ ]:
four_columns_for_activity = [
    'Case ID',
    'label',
    'Activity',
    'time:timestamp'
]


## Activity data preparation

In [ ]:
activity_data = unprepared_data[four_columns_for_activity]
activity_data.head()

In [ ]:
activity_data['label'] = np.where(activity_data['label'] == 'deviant', 1, 0)
activity_data.head()

In [ ]:
activity_data = activity_data.rename(columns={'label': 'Outcome', 'time:timestamp': 'Timestamp'})
activity_data.head()

In [ ]:
activity_data.to_csv(DESTINATION_DATA, index=False)

In [ ]:
len(activity_data['Activity'].unique())

In [ ]:
np.mean(activity_data.groupby('Case ID').count()['Activity'])

## Numeric data preparation for comparison with Bozorgi's et al. study: https://arxiv.org/abs/2009.01561

In [ ]:
numeric_cols = other_columns + numeric_columns
numeric_cols

In [ ]:
numeric_data = unprepared_data[numeric_cols]
numeric_data.head()

In [ ]:
numeric_data['label'] = np.where(numeric_data['label'] == 'deviant', 1, 0)
numeric_data.head()

In [ ]:
numeric_data = numeric_data.rename(columns={'label': 'Outcome', 'time:timestamp': 'Timestamp'})
numeric_data.head()

In [ ]:
numeric_data = numeric_data.groupby('Case ID').apply(lambda row: row.max())
numeric_data = numeric_data.reset_index(drop=True)
numeric_data.head()

In [ ]:
del numeric_data['Case ID']
numeric_data.head()

In [ ]:
numeric_data['NumberOfTerms_treatment'] = numeric_data['NumberOfTerms']
numeric_data['FirstWithdrawalAmount_treatment'] = numeric_data['FirstWithdrawalAmount']
numeric_data['NumberOfTerms_control'] = numeric_data['NumberOfTerms']
numeric_data['FirstWithdrawalAmount_control'] = numeric_data['FirstWithdrawalAmount']
numeric_data.head()

In [ ]:
def transform(x, A, B, C, D):
    return (D * (x - A) + C * (B - x)) / (B - A)

In [ ]:
def apply_to_row(row):
    action_1 = (row['FirstWithdrawalAmount'] > 7500) & \
               (row['FirstWithdrawalAmount'] <= 9895) & \
               (row['ApplicationType'] == 'Limit raise') & \
               (row['CreditScore'] > 885) & \
               (row['MonthlyCost'] > 120)

    action_2 = (row['NumberOfTerms'] > 6) & \
               (row['NumberOfTerms'] <= 48) & \
               (row['CreditScore'] > 899) & \
               (row['CreditScore'] <= 943) & \
               (row['FirstWithdrawalAmount'] < 8304)

    action_3 = (row['NumberOfTerms'] > 6) & \
               (row['NumberOfTerms'] <= 48) & \
               (row['LoanGoal'] != 'Existing loan takeover') & \
               (row['MonthlyCost'] > 149) & \
               (row['FirstWithdrawalAmount'] < 8304)

    action_4 = (row['NumberOfTerms'] > 6) & \
               (row['NumberOfTerms'] <= 48) & \
               (
                       (
                               (row['CreditScore'] < 982) &
                               (row['FirstWithdrawalAmount'] > 8304) &
                               (row['MonthlyCost'] > 154) &
                               (row['MonthlyCost'] <= 205)
                       ) |
                       (
                               (row['CreditScore'] > 781) &
                               (row['CreditScore'] <= 982) &
                               (row['FirstWithdrawalAmount'] < 8304) &
                               (row['MonthlyCost'] > 147)
                       )
               )

    action_5 = (row['FirstWithdrawalAmount'] > 7500) & \
               (row['FirstWithdrawalAmount'] <= 9895) & \
               (row['NumberOfTerms'] > 120) & \
               (row['MonthlyCost'] < 150)

    action_6 = (row['FirstWithdrawalAmount'] > 7500) & \
               (row['FirstWithdrawalAmount'] <= 9895) & \
               (row['NumberOfTerms'] > 120) & \
               (row['ApplicationType'] == 'New credit') & \
               (row['CreditScore'] < 914) & \
               (row['MonthlyCost'] > 150)

    action_7 = (row['FirstWithdrawalAmount'] > 7500) & \
               (row['FirstWithdrawalAmount'] <= 9895) & \
               (row['ApplicationType'] == 'New credit') & \
               (row['LoanGoal'] == 'Existing loan takeover') & \
               (row['CreditScore'] == 825)

    action_8 = (row['FirstWithdrawalAmount'] > 9895) & \
               (row['FirstWithdrawalAmount'] <= 75000) & \
               (row['NumberOfTerms'] > 49) & \
               (row['NumberOfTerms'] <= 60) & \
               (row['CreditScore'] < 933) & \
               (row['MonthlyCost'] > 154)

    if action_1:
        row['FirstWithdrawalAmount_treatment'] = transform(row['FirstWithdrawalAmount'], 7500, 9895, 0, 7499)
    if action_2:
        row['NumberOfTerms_treatment'] = transform(row['NumberOfTerms'], 6, 48, 120, 360)
    if action_3:
        row['NumberOfTerms_treatment'] = transform(row['NumberOfTerms'], 6, 48, 61, 96)
    if action_4:
        row['NumberOfTerms_treatment'] = transform(row['NumberOfTerms'], 6, 48, 97, 120)
    if action_5:
        row['FirstWithdrawalAmount_treatment'] = transform(row['FirstWithdrawalAmount'], 7500, 9895,  5750, 7499)
        row['NumberOfTerms_treatment'] = transform(row['NumberOfTerms'], 120, 600, 49, 60)
    if action_6:
        row['FirstWithdrawalAmount_treatment'] = transform(row['FirstWithdrawalAmount'], 7500, 9895,  0, 7499)
        row['NumberOfTerms_treatment'] = transform(row['NumberOfTerms'], 120, 600, 97, 120)
    if action_7:
        row['FirstWithdrawalAmount_treatment'] = transform(row['FirstWithdrawalAmount'], 7500, 9895,  9896, 75000)
    if action_8:
        row['FirstWithdrawalAmount_treatment'] = transform(row['FirstWithdrawalAmount'], 9896, 75000,  1490, 7499)
        row['NumberOfTerms_treatment'] = transform(row['NumberOfTerms'], 49, 60, 97, 120)
    return row


In [ ]:
numeric_data = numeric_data.apply(apply_to_row, axis=1)
numeric_data.head()

In [ ]:
numeric_data['ApplicationType'] = np.where(numeric_data['ApplicationType'] == 'New credit', 1.0, 0.0)
numeric_data['LoanGoal'] = np.where(numeric_data['LoanGoal'] == 'Existing loan takeover', 1.0, 0.0)
numeric_data.head()

In [ ]:
train, test = train_test_split(numeric_data, test_size=0.2, random_state=42)


In [ ]:
train.to_csv(NUMERIC_TRAIN, index=False)
test.to_csv(NUMERIC_TEST, index=False)

